In [13]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

In [18]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

#get test data
train_x = Variable(torch.unsqueeze(train_dataset.data,dim=1)).type(torch.FloatTensor)[2000:]/255.
train_y = train_dataset.targets[2000:]
test_x = Variable(torch.unsqueeze(test_dataset.data,dim=1)).type(torch.FloatTensor)[:2000]/255.
test_y = test_dataset.targets[:2000]


In [19]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(    #(1,28,28)1是channel的维度，28*28为图片的长宽
                in_channels=1,#图片的层数，RGB=3,灰度=1
                out_channels=16,#filter的个数
                kernel_size=5,#filter的长宽
                stride=1,#每隔多少个移动
                padding=2,#图片补0.if stride = 1,padding =(kernel_size-1)/2 = (5-1)/2 = 2
            ),   #-->（16，28，28）
            nn.ReLU(),#-->（16，28，28）
            nn.MaxPool2d(kernel_size=2),#可以看成2*2的filter    #-->（16，14，14）kernel_size = 2,减小一半
        )
        self.conv2 = nn.Sequential(#-->（16，14，14）
            nn.Conv2d(16,32,5,1,2),#-->（32，14，14）
            nn.ReLU(),#-->（32，14，14）
            nn.MaxPool2d(2)#-->（32，7，7）

        )
        self.out = nn.Linear(32*7*7,10)#输出为10类

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)   #-->(batch,32,7,7)
        x = x.view(x.size(0),-1) #-->(batch,32*7*7)
        output = self.out(x)
        return output


    
model = SimpleNet()
print(model)
# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

SimpleNet(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


In [21]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images,labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        b_x = Variable(images)
        b_y = Variable(labels)

        out = model(b_x)
        loss = criterion(out,b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        
    test_out = model(test_x)
    train_out = model(train_x)
    pred_train_y = torch.max(train_out,1)[1].data.squeeze()
    pred_test_y = torch.max(test_out,1)[1].data.squeeze()
    
    accuracy_train = float(sum(pred_train_y == train_y)) / float(sum(train_y == train_y))    
    accuracy_test = float(sum(pred_test_y == test_y))/ float(sum(test_y == test_y))
    
    print('Epoch:',epoch,'|train accuracy:'+str(accuracy_train),'|test accuracy:'+str(accuracy_test))

        
        
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset


100%|██████████| 468/468 [00:42<00:00, 10.91it/s]

  0%|          | 0/468 [00:00<?, ?it/s]

Epoch: 0 |train accuracy:0.9730689655172414 |test accuracy:0.967



100%|██████████| 468/468 [00:47<00:00,  9.93it/s]

  0%|          | 0/468 [00:00<?, ?it/s]

Epoch: 1 |train accuracy:0.9592586206896552 |test accuracy:0.9445



100%|██████████| 468/468 [00:44<00:00, 10.45it/s]

  0%|          | 0/468 [00:00<?, ?it/s]

...

Epoch: 7 |train accuracy:0.9287758620689656 |test accuracy:0.899



100%|██████████| 468/468 [00:44<00:00, 10.63it/s]

  0%|          | 0/468 [00:00<?, ?it/s]

Epoch: 8 |train accuracy:0.7829482758620689 |test accuracy:0.7315



100%|██████████| 468/468 [00:44<00:00, 10.54it/s]


Epoch: 9 |train accuracy:0.9527758620689655 |test accuracy:0.9345
